# 初始化階段

### 捷運站點 graph

In [4]:
import csv
import re

def read_csv_to_array(file_path):
    data = []
    with open(file_path, 'r', encoding = "utf-8") as file:
        csv_reader = csv.reader(file)
        for row in csv_reader:
            data.append(row)
    return data

def extract_numbers(text):
    number_pattern = r'\d+'
    numbers = re.findall(number_pattern, text)
    return numbers

def extract_english(text):
    english_pattern = r'[a-zA-Z]+'
    english_words = re.findall(english_pattern, text)
    english_text = ' '.join(english_words)
    return english_text

def extract_chinese(text):
    chinese_pattern = r'[\u4e00-\u9fff]+'
    chinese_words = re.findall(chinese_pattern, text)
    chinese_text = ' '.join(chinese_words)
    return chinese_text

def number_str_computer(text = "", x = 0):
    n = int(text) + x
    return str(n).zfill(2)


file_path = '捷運票價.csv'
csv_data = read_csv_to_array(file_path)
graph = {}

# for i in range(1, len(csv_data)):
#     graph[csv_data[i][0]] = []
#     number = []
#     for j in range(1, len(csv_data)):
#         if extract_chinese(csv_data[j][0]) == extract_chinese(csv_data[i][0]):
#             number.append(csv_data[j][0])
    
#     for comp in number:
#         for j in range(1, len(csv_data)):
#             if extract_english(csv_data[j][0]) == extract_english(comp) and (number_str_computer(extract_numbers(csv_data[j][0])[0], 1) == extract_numbers(comp)[0] or number_str_computer(extract_numbers(csv_data[j][0])[0], -1) == extract_numbers(comp)[0]):
#                 graph[csv_data[i][0]].append(csv_data[j][0])

for i in range(1, len(csv_data)):
    graph[csv_data[i][0]] = []
    for j in range(1, len(csv_data)):
        if extract_chinese(csv_data[j][0]) == extract_chinese(csv_data[i][0]) and csv_data[j][0] != csv_data[i][0]:
            graph[csv_data[i][0]].append(csv_data[j][0])
            
        if extract_english(csv_data[j][0]) == extract_english(csv_data[i][0]) and (number_str_computer(extract_numbers(csv_data[j][0])[0], 1) == extract_numbers(csv_data[i][0])[0] or number_str_computer(extract_numbers(csv_data[j][0])[0], -1) == extract_numbers(csv_data[i][0])[0]):
            graph[csv_data[i][0]].append(csv_data[j][0])
                
graph['R22 北投'].append('R22A 新北投')
graph['R22A 新北投'].append('R22 北投')
graph['G03 七張'].append('G03A 小碧潭')
graph['G03A 小碧潭'].append('G03 七張')
graph['O12 大橋頭'].append('O50 三重國小')
graph['O50 三重國小'].append('O12 大橋頭')

In [30]:
import csv
import xml.etree.ElementTree as ET

tree = ET.parse('text.xml')
xml_root = tree.getroot()
with open('output.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)

    csvwriter.writerow(["起點站", "終點站", "行車時間", "停等時間"])
    for travelTime in xml_root.findall('TravelTime'):
        from_station = travelTime.find('FromStationName/Zh_tw').text
        to_station = travelTime.find('ToStationName/Zh_tw').text
        run_time = (travelTime.find('RunTime').text)
        stop_time = (travelTime.find('StopTime').text)

        # csvwriter.writerow([from_station, to_station, run_time, stop_time])
        print([from_station, to_station, run_time, stop_time])


['南港展覽館', '南港', '112', '0']
['南港', '昆陽', '109', '24']
['昆陽', '後山埤', '107', '25']
['後山埤', '永春', '76', '29']
['永春', '市政府', '87', '25']
['市政府', '國父紀念館', '77', '28']
['國父紀念館', '忠孝敦化', '71', '28']
['忠孝敦化', '忠孝復興', '65', '28']
['忠孝復興', '忠孝新生', '90', '40']
['忠孝新生', '善導寺', '81', '40']
['善導寺', '台北車站', '66', '25']
['台北車站', '西門', '120', '42']
['西門', '龍山寺', '110', '35']
['龍山寺', '江子翠', '192', '29']
['江子翠', '新埔', '77', '28']
['新埔', '板橋', '107', '28']
['板橋', '府中', '67', '28']
['府中', '亞東醫院', '102', '28']
['亞東醫院', '海山', '142', '26']
['海山', '土城', '106', '26']
['土城', '永寧', '95', '26']
['永寧', '頂埔', '180', '28']


### 捷運終點站

In [6]:
station_last_dic = {
    "BR" : ["BR 文湖線","動物園","南港展覽館"],
    "R" : ["R 淡水信義線","象山","淡水"], 
    "O" : ["O 中和新盧線","南勢角","蘆洲、迴龍","迴龍", "蘆洲"], 
    "BL" : ["BL 板南線","頂埔","南港展覽館"], 
    "G" : ["G 松山新店線","新店","松山"], 
    "Y" : ["Y 環狀線","大坪林","新北產業園區"], 
}

def find_last_station(station, next_station, path):
    if "A" in extract_english(station) or "A" in extract_english(next_station):
        return "轉乘支線"
        
    elif "O" in extract_english(station):
        if int(extract_numbers(station)[0]) > int(extract_numbers(next_station)[0]):
            return "往" + station_last_dic[extract_english(station)][1]
        else:
            if "O13 台北橋" in path or "O15 三重" in path or "O16 先嗇宮" in path or "O18 新莊" in path or "O20 丹鳳" in path:
                return "往" + station_last_dic[extract_english(station)][3]
            elif "O50 三重國小" in path or "O52 徐匯中學" in path or "O54 蘆洲" in path:
                return "往" + station_last_dic[extract_english(station)][4]
            else:
                return "往" + station_last_dic[extract_english(station)][2]
            
    else:
        if int(extract_numbers(station)[0]) > int(extract_numbers(next_station)[0]):
            return "往" + station_last_dic[extract_english(station)][1]
        else:
            return "往" + station_last_dic[extract_english(station)][2]

### 捷運時刻表

### 演算法 function

In [7]:
from collections import deque

def find_all_paths(graph, start, end): # 建立所有路徑
    paths = []
    queue = deque([(start, [start])])

    while queue:
        current_node, path = queue.popleft()

        if current_node == end:
            paths.append(path)
        else:
            for neighbor in graph[current_node]:
                if neighbor not in path:
                    queue.append((neighbor, path + [neighbor]))

    return paths

# 0:站數; 1:轉乘數; 2:經過路線; 3:所需時間
def get_paths_info(paths): # 計算所有路徑的屬性
    paths_info = []
    for p in range(0, len(paths)):
        tmpList = [0, 0, [], 0]
        if "O13 台北橋" in paths[p] and "O50 三重國小" in paths[p]:
            tmpList[1] += 1
        for i in range(1, len(paths[p])):
            if extract_chinese(paths[p][i-1]) != extract_chinese(paths[p][i]):
                tmpList[0] += 1
            elif i > 1 and i < len(paths[p])-1:
                tmpList[1] += 1
                if extract_english(paths[p][i-1]) not in tmpList[2]:
                    tmpList[2].append(extract_english(paths[p][i-1]))
                if extract_english(paths[p][i]) not in tmpList[2]:
                    tmpList[2].append(extract_english(paths[p][i]))

        if len(tmpList[2]) == 0:
            if extract_chinese(paths[p][0]) == extract_chinese(paths[p][1]):
                tmpList[2].append(extract_english(paths[p][1]))
            else:
                tmpList[2].append(extract_english(paths[p][0]))
                
        pre_station_index = 0
        for i in range(1, len(paths[p])):
            if extract_chinese(paths[p][i]) == extract_chinese(paths[p][i-1]):
                tmp_info = csv_data[csv_data[0].index(paths[p][pre_station_index])][csv_data[0].index(paths[p][i])].split("/")
                if len(tmp_info) > 2:
                    tmpList[3] += int(tmp_info[4])
                pre_station_index = i
        
        tmp_info = csv_data[csv_data[0].index(paths[p][pre_station_index])][csv_data[0].index(paths[p][-1])].split("/")
        if len(tmp_info) > 2:
            tmpList[3] += int(tmp_info[4])

        paths_info.append(tmpList)


    return paths_info

# 圖形化介面執行

### 前置 function

In [8]:
# 文字輸出
from colorama import init, Fore, Style
def __print_station_color(station, endchar):
    if extract_english(station) == "BR":
        text_box.tag_config("brown", foreground="#ab7722")
        text_box.insert("end", station, "brown")
    elif extract_english(station) == "R" or extract_english(station) == "R A":
        text_box.tag_config("red", foreground="#e3002c")
        text_box.insert("end", station, "red")
    elif extract_english(station) == "G" or extract_english(station) == "G A":
        text_box.tag_config("green", foreground="#008659")
        text_box.insert("end", station, "green")
    elif extract_english(station) == "O":
        text_box.tag_config("orange", foreground="#ffb914")
        text_box.insert("end", station, "orange")
    elif extract_english(station) == "BL":
        text_box.tag_config("blue", foreground="#0070bd")
        text_box.insert("end", station, "blue")
    elif extract_english(station) == "Y":
        text_box.tag_config("yellow", foreground="#f2cf11")
        text_box.insert("end", station, "yellow")
    
    text_box.insert("end", endchar)
    return 0
        
def print_path(list):
    if "O13 台北橋" in list and "O50 三重國小" in list:
        if extract_chinese(list[0]) == extract_chinese(list[1]):
            __print_station_color(list[1], "")
        else:
            __print_station_color(list[0], "")

        text_box.tag_configure("italic", font=("Arial", 12, "roman"))
        text_box.insert("end", "（往南勢角）\t→\n", "italic")
        __print_station_color("O12 大橋頭", "")
        text_box.tag_configure("italic", font=("Arial", 12, "roman"))
        if list[len(list)-2] != "O17 頭前庄":
            text_box.insert("end", "（" + find_last_station(list[len(list)-2], list[len(list)-1], list) + "）\t→\n", "italic")
            __print_station_color(list[len(list)-1], "\n\n")
        else:
            text_box.insert("end", "（" + find_last_station(list[len(list)-3], list[len(list)-2], list) + "）\t→\n", "italic")
            __print_station_color(list[len(list)-2], "\n\n")
        return
    
    if len(list) == 2:
        __print_station_color(list[0], "")
        text_box.tag_configure("italic", font=("Arial", 12, "roman"))
        text_box.insert("end", "（" + find_last_station(list[0], list[1], list) + "）", "italic")
        text_box.insert("end", "\t→\n")
        __print_station_color(list[1], "\n\n")
        return
        
    if extract_chinese(list[0]) == extract_chinese(list[1]):
        __print_station_color(list[1], "")
        text_box.tag_configure("italic", font=("Arial", 12, "roman"))
        text_box.insert("end", "（" + find_last_station(list[1], list[2], list) + "）", "italic")
        text_box.insert("end", "\t→\n")
    else:
        __print_station_color(list[0], "")
        text_box.tag_configure("italic", font=("Arial", 12, "roman"))
        text_box.insert("end", "（" + find_last_station(list[0], list[1], list) + "）", "italic")
        text_box.insert("end", "\t→\n")

    if 'A' in extract_english(list[0]):
        __print_station_color(list[1], "")
        text_box.tag_configure("italic", font=("Arial", 12, "roman"))
        text_box.insert("end", "（" + find_last_station(list[1], list[2], list) + "）", "italic")
        text_box.insert("end", "\t→\n")
    
    for i in range(2, len(list)-1):
        if extract_chinese(list[i-1]) == extract_chinese(list[i]):
            __print_station_color(list[i-1].split(" ")[0] + "／", "")
            __print_station_color(list[i], "")
            
            text_box.tag_configure("italic", font=("Arial", 12, "roman"))
            text_box.insert("end", "（" + find_last_station(list[i], list[i+1], list) + "）", "italic")
            
            text_box.insert("end", "\t→\n")

    if 'A' in extract_english(list[-1]):
        __print_station_color(list[-2], "")
        text_box.tag_configure("italic", font=("Arial", 12, "roman"))
        text_box.insert("end", "（" + find_last_station(list[-2], list[-1], list) + "）", "italic")
        text_box.insert("end", "\t→\n")
        __print_station_color(list[-1], "\n\n")
        return
    
    if extract_chinese(list[len(list)-2]) == extract_chinese(list[len(list)-1]):
        __print_station_color(list[len(list)-2], "\n\n")
    else:
        __print_station_color(list[len(list)-1], "\n\n")
        
    return

# 圖形輸出
# R 是半徑; 0: x, 1: y, 2: 終點站(1)與轉運站(2)
r = 6
MRT_circle = {
    "BR01 動物園":[383, 472, 1],
    "BR02 木柵":[362, 472, 0],
    "BR03 萬芳社區":[340, 461, 0],
    "BR04 萬芳醫院":[340, 440, 0],
    "BR05 辛亥":[340, 419, 0],
    "BR06 麟光":[323, 403, 0],
    "BR07 六張犁":[301, 403, 0],
    "BR08 科技大樓":[280, 387, 0],
    "BR09 大安":[280, 361, 0],
    "BR10 忠孝復興":[280, 319, 0],
    "BR11 南京復興":[280, 288, 0],
    "BR12 中山國中":[280, 261, 0],
    "BR13 松山機場":[280, 240, 0],
    "BR14 大直":[280, 219, 0],
    "BR15 劍南路":[280, 198, 0],
    "BR16 西湖":[308, 198, 0],
    "BR17 港墘":[336, 198, 0],
    "BR18 文德":[364, 198, 0],
    "BR19 內湖":[389, 198, 0],
    "BR20 大湖公園":[410, 198, 0],
    "BR21 葫洲":[431, 198, 0],
    "BR22 東湖":[431, 219, 0],
    "BR23 南港軟體園區":[431, 240, 0],
    "BR24 南港展覽館":[431, 261, 1],
    "R02 象山":[370, 361, 1],
    "R03 台北101/世貿":[340, 361, 0],
    "R04 信義安和":[310, 361, 0],
    "R05 大安":[280, 361, 2],
    "R06 大安森林公園":[256, 361, 0],
    "R07 東門":[232, 361, 0],
    "R08 中正紀念堂":[190, 361, 0],
    "R09 台大醫院":[190, 340, 0],
    "R10 台北車站":[190, 319, 0],
    "R11 中山":[190, 288, 0],
    "R12 雙連":[190, 267, 0],
    "R13 民權西路":[190, 246, 0],
    "R14 圓山":[190, 225, 0],
    "R15 劍潭":[190, 204, 0],
    "R16 士林":[190, 183, 0],
    "R17 芝山":[190, 162, 0],
    "R18 明德":[190, 141, 0],
    "R19 石牌":[190, 120, 0],
    "R20 唭哩岸":[174, 104, 0],
    "R21 奇岩":[153, 104, 0],
    "R22 北投":[132, 104, 0],
    "R22A 新北投":[132, 74, 0],
    "R23 復興崗":[111, 104, 0],
    "R24 忠義":[90, 104, 0],
    "R25 關渡":[74, 88, 0],
    "R26 竹圍":[74, 67, 0],
    "R27 紅樹林":[74, 46, 0],
    "R28 淡水":[74, 24, 1],
    "G01 新店":[305, 543, 1],
    "G02 新店區公所":[305, 522, 0],
    "G03 七張":[305, 501, 0],
    "G03A 小碧潭":[275, 501, 0],
    "G04 大坪林":[305, 480, 0],
    "G05 景美":[287, 456, 0],
    "G06 萬隆":[270, 437, 0],
    "G07 公館":[250, 418, 0],
    "G08 台電大樓":[230, 399, 0],
    "G09 古亭":[210, 380, 0],
    "G10 中正紀念堂":[190, 361, 2],
    "G11 小南門":[160, 361, 0],
    "G12 西門":[160, 319, 0],
    "G13 北門":[160, 288, 0],
    "G14 中山":[190, 288, 2],
    "G15 松江南京":[232, 288, 0],
    "G16 南京復興":[280, 288, 2],
    "G17 台北小巨蛋":[310, 288, 0],
    "G18 南京三民":[340, 288, 0],
    "G19 松山":[370, 288, 1],
    "O01 南勢角":[199, 480, 1],
    "O02 景安":[199, 456, 0],
    "O03 永安市場":[199, 432, 0],
    "O04 頂溪":[199, 408, 0],
    "O05 古亭":[210, 380, 2],
    "O06 東門":[232, 361, 2],
    "O07 忠孝新生":[232, 319, 0],
    "O08 松江南京":[232, 288, 2],
    "O09 行天宮":[232, 262, 0],
    "O10 中山國小":[216, 246, 0],
    "O11 民權西路":[190, 246, 2],
    "O12 大橋頭":[160, 246, 0],
    "O13 台北橋":[120, 262, 0],
    "O14 菜寮":[100, 278, 0],
    "O15 三重":[80, 294, 0],
    "O16 先嗇宮":[60, 310, 0],
    "O17 頭前庄":[40, 326, 0],
    "O18 新莊":[20, 342, 0],
    "O19 輔大":[20, 363, 0],
    "O20 丹鳳":[20, 384, 0],
    "O21 迴龍":[20, 405, 1],
    "O50 三重國小":[120, 230, 0],
    "O51 三和國中":[100, 214, 0],
    "O52 徐匯中學":[100, 193, 0],
    "O53 三民高中":[100, 172, 0],
    "O54 蘆洲":[100, 151, 1],
    "BL01 頂埔":[92, 507, 1],
    "BL02 永寧":[92, 486, 0],
    "BL03 土城":[92, 465, 0],
    "BL04 海山":[92, 444, 0],
    "BL05 亞東醫院":[92, 423, 0],
    "BL06 府中":[92, 402, 0],
    "BL07 板橋":[92, 378, 0],
    "BL08 新埔":[91, 360, 0],
    "BL09 江子翠":[92, 339, 0],
    "BL10 龍山寺":[120, 319, 0],
    "BL11 西門":[160, 319, 2],
    "BL12 台北車站":[190, 319, 2],
    "BL13 善導寺":[211, 319, 0],
    "BL14 忠孝新生":[232, 319, 2],
    "BL15 忠孝復興":[280, 319, 2],
    "BL16 忠孝敦化":[310, 319, 0],
    "BL17 國父紀念館":[340, 319, 0],
    "BL18 市政府":[365, 319, 0],
    "BL19 永春":[390, 319, 0],
    "BL20 後山埤":[415, 319, 0],
    "BL21 昆陽":[431, 303, 0],
    "BL22 南港":[431, 282, 0],
    "BL23 南港展覽館":[431, 261, 2],
    "Y07 大坪林":[305, 480, 1],
    "Y08 十四張":[275, 480, 0],
    "Y09 秀朗橋":[250, 468, 0],
    "Y10 景平":[224, 457, 0],
    "Y11 景安":[199, 457, 2],
    "Y12 中和":[174, 457, 0],
    "Y13 橋和":[155, 435, 0],
    "Y14 中原":[134, 416, 0],
    "Y15 板新":[113, 397, 0],
    "Y16 板橋":[92, 378, 2],
    "Y17 新埔民生":[76, 360, 0],
    "Y18 頭前庄":[40, 326, 2],
    "Y19 幸福":[20, 305, 0],
    "Y20 新北產業園區":[20, 279, 1]
}


def __get_circle_bg_color(station):
    if extract_english(station) == "BR":
        return "#e5d7c0"
    elif extract_english(station) == "R" or extract_english(station) == "R A":
        return "#edb4bf"
    elif extract_english(station) == "G" or extract_english(station) == "G A":
        return "#b7d6cb"
    elif extract_english(station) == "O":
        return "#f2e1ba"
    elif extract_english(station) == "BL":
        return "#b4d0e3"
    elif extract_english(station) == "Y":
        return "#f3ebba"

def __get_circle_color(station):
    if extract_english(station) == "BR":
        return "#ab7722"
    elif extract_english(station) == "R" or extract_english(station) == "R A":
        return "#e3002c"
    elif extract_english(station) == "G" or extract_english(station) == "G A":
        return "#008659"
    elif extract_english(station) == "O":
        return "#ffb914"
    elif extract_english(station) == "BL":
        return "#0070bd"
    elif extract_english(station) == "Y":
        return "#f2cf11"
    
def draw_bg():
    global MRT_circle
    global r
    canvas.delete('all')
    for key, value in MRT_circle.items():
        if len(value) > 0:
            if value[2] == 1:
                canvas.create_oval(value[0]-r-1, value[1]-r-1, value[0]+r+1, value[1]+r+1, outline=__get_circle_bg_color(key), fill=__get_circle_bg_color(key))
            elif value[2] == 2:
                canvas.create_oval(value[0]-r+2, value[1]-r+2, value[0]+r-2, value[1]+r-2, fill=__get_circle_bg_color(key), width=0)
            else:
                canvas.create_oval(value[0]-r, value[1]-r, value[0]+r, value[1]+r, outline=__get_circle_bg_color(key), width=4)

            for neighbor in graph[key]:
                canvas.create_line(value[0], value[1], MRT_circle[neighbor][0], MRT_circle[neighbor][1], fill=__get_circle_bg_color(key), width=3)

def __get_xy(pre_x, pre_y, x, y, next_x, next_y, text_count):
    # 0 1 2
    # 3   4
    # 5 6 7
    dic = [0]*8

    if (pre_x < x and pre_y < y) or (next_x < x and next_y < y):
        dic[0] = 1
    if (pre_x == x and pre_y < y) or (next_x == x and next_y < y):
        dic[1] = 1
    if (pre_x > x and pre_y < y) or (next_x > x and next_y < y):
        dic[2] = 1
    if (pre_x < x and pre_y == y) or (next_x < x and next_y == y):
        dic[3] = 1
    if (pre_x > x and pre_y == y) or (next_x > x and next_y == y):
        dic[4] = 1
    if (pre_x < x and pre_y > y) or (next_x < x and next_y > y):
        dic[5] = 1
    if (pre_x == x and pre_y > y) or (next_x == x and next_y > y):
        dic[6] = 1
    if (pre_x > x and pre_y > y) or (next_x > x and next_y > y):
        dic[7] = 1

    count = 0
    for n in dic:
        count += n
    
    if count == 0:
        return[0, 0]
    
    k = text_count*7 + 8
    if dic[1] == 0 and dic[2] == 0 and dic[4] == 0:
        return[x+k+2, y-10]
    elif dic[0] == 0 and dic[1] == 0 and dic[3] == 0:
        return[x-k, y-10]
    elif dic[5] == 0 and dic[6] == 0 and dic[7] == 0:
        return[x, y+18]
    elif dic[4] == 0 and dic[6] == 0 and dic[7] == 0:
        return[x+k+2, y+10]
    elif dic[3] == 0 and dic[5] == 0 and dic[6] == 0:
        return[x-k, y+10]
    elif dic[4] == 0:
        return[x+k+2, y]
    else:
        return[x-k, y]
    


def draw_path(path):
    global MRT_circle
    global r
    draw_bg()
    xy = __get_xy(0, 0, MRT_circle[path[0]][0], MRT_circle[path[0]][1], MRT_circle[path[1]][0], MRT_circle[path[1]][1], len(path[0].split(" ")[1]))
    if xy[0] == 0 and xy[1] == 0:
        xy = __get_xy(0, 0, MRT_circle[path[1]][0], MRT_circle[path[1]][1], MRT_circle[path[2]][0], MRT_circle[path[2]][1], len(path[0].split(" ")[1]))

    canvas.create_text(xy[0], xy[1], text=path[0].split(" ")[1], font=("Arial", 10))
    canvas.create_oval(MRT_circle[path[0]][0]-r-1, MRT_circle[path[0]][1]-r-1, MRT_circle[path[0]][0]+r+1, MRT_circle[path[0]][1]+r+1, fill=__get_circle_color(path[0]), width=0)

    if "O13 台北橋" in path and "O50 三重國小" in path:
        canvas.create_text(168, 262, text="大橋頭", font=("Arial", 10))

    for i in range(1, len(path)):
        if extract_chinese(path[i]) == extract_chinese(path[i-1]):
            canvas.create_oval(MRT_circle[path[i]][0]-r+2, MRT_circle[path[i]][1]-r+2, MRT_circle[path[i]][0]+r-2, MRT_circle[path[i]][1]+r-2, fill=__get_circle_color(path[i]), width=0)
            if i == 1:
                canvas.create_oval(MRT_circle[path[i]][0]-r-1, MRT_circle[path[i]][1]-r-1, MRT_circle[path[i]][0]+r+1, MRT_circle[path[i]][1]+r+1, fill=__get_circle_color(path[i]), width=0)
            elif i == len(path) - 1:
                canvas.create_oval(MRT_circle[path[i-1]][0]-r-1, MRT_circle[path[i-1]][1]-r-1, MRT_circle[path[i-1]][0]+r+1, MRT_circle[path[i-1]][1]+r+1, fill=__get_circle_color(path[i-1]), width=0)
            else:
                xy = __get_xy(MRT_circle[path[i-2]][0], MRT_circle[path[i-2]][1], MRT_circle[path[i]][0], MRT_circle[path[i]][1], MRT_circle[path[i+1]][0], MRT_circle[path[i+1]][1], len(path[i].split(" ")[1]))
                canvas.create_text(xy[0], xy[1], text=path[i].split(" ")[1], font=("Arial", 10))
        else:
            if i == len(path) - 1:
                canvas.create_oval(MRT_circle[path[i]][0]-r-1, MRT_circle[path[i]][1]-r-1, MRT_circle[path[i]][0]+r+1, MRT_circle[path[i]][1]+r+1, fill=__get_circle_color(path[i-1]), width=0)
            else:
                canvas.create_oval(MRT_circle[path[i]][0]-r, MRT_circle[path[i]][1]-r, MRT_circle[path[i]][0]+r, MRT_circle[path[i]][1]+r, outline=__get_circle_color(path[i]), width=4)

        canvas.create_line(MRT_circle[path[i-1]][0], MRT_circle[path[i-1]][1], MRT_circle[path[i]][0], MRT_circle[path[i]][1], fill=__get_circle_color(path[i]), width=3)

    xy = __get_xy(0, 0, MRT_circle[path[-1]][0], MRT_circle[path[-1]][1], MRT_circle[path[-2]][0], MRT_circle[path[-2]][1], len(path[-1].split(" ")[1]))
    if xy[0] == 0 and xy[1] == 0:
        xy = __get_xy(0, 0, MRT_circle[path[-2]][0], MRT_circle[path[-2]][1], MRT_circle[path[-3]][0], MRT_circle[path[-3]][1], len(path[-1].split(" ")[1]))
    canvas.create_text(xy[0], xy[1], text=path[-1].split(" ")[1], font=("Arial", 10))

    # for key, value in MRT_circle.items():
    #     if len(value) > 0 and key in path:
    #         if value[2] == 1:
    #             canvas.create_oval(value[0]-r-1, value[1]-r-1, value[0]+r+1, value[1]+r+1, outline=__get_circle_color(key), fill=__get_circle_color(key))
    #         elif value[2] == 2:
    #             canvas.create_oval(value[0]-r+2, value[1]-r+2, value[0]+r-2, value[1]+r-2, fill=__get_circle_color(key), width=0)
    #         else:
    #             canvas.create_oval(value[0]-r, value[1]-r, value[0]+r, value[1]+r, outline=__get_circle_color(key), width=4)

    #         for neighbor in graph[key]:
    #             if neighbor in path:
    #                 canvas.create_line(value[0], value[1], MRT_circle[neighbor][0], MRT_circle[neighbor][1], fill=__get_circle_color(key), width=3)

# 檢索路徑
def get_road():
    text_box.delete("1.0", "end")
    if combobox_start.get() == '請選擇站點' or  combobox_end.get() == '請選擇站點':
        text_box.insert("end", "尚未選擇站點")
        draw_bg()
        return
    if extract_chinese(combobox_start.get()) == extract_chinese(combobox_end.get()):
        text_box.insert("end", "同站進出： ")
        if combobox_start.get() != combobox_end.get():
            __print_station_color(combobox_start.get().split(" ")[0] + "/", "")
        __print_station_color(combobox_end.get(), "\n")
        global MRT_circle
        global r
        draw_bg()
        canvas.create_oval(MRT_circle[combobox_start.get()][0]-r-1, MRT_circle[combobox_start.get()][1]-r-1, MRT_circle[combobox_start.get()][0]+r+1, MRT_circle[combobox_start.get()][1]+r+1, fill=__get_circle_color(combobox_start.get()), width=0)
        canvas.create_text(MRT_circle[combobox_end.get()][0], MRT_circle[combobox_end.get()][1]+18, text=combobox_end.get().split(" ")[1], font=("Arial", 10))
        return
    
    global paths
    global info
    global paths_info
    
    res = []
    time_spand = 0
    max_list = [99999, 99999, 0, 99999]
    
    for i in range(0, len(paths)):  
        if (not checkbox_var_BR.get()) and "BR" in paths_info[i][2]:
            continue
        if (not checkbox_var_R.get()) and "R" in paths_info[i][2]:
            continue
        if (not checkbox_var_G.get()) and "G" in paths_info[i][2]:
            continue
        if (not checkbox_var_O.get()) and "O" in paths_info[i][2]:
            continue
        if (not checkbox_var_BL.get()) and "BL" in paths_info[i][2]:
            continue
        if (not checkbox_var_Y.get()) and "Y" in paths_info[i][2]:
            continue
            
        tmp_list = [3, 1, 0]
        if combobox_sort.current() == 1:
            tmp_list = [1, 3, 0]
        elif combobox_sort.current() == 2:
            tmp_list = [0, 3, 1]
        
        if paths_info[i][tmp_list[0]] < max_list[tmp_list[0]]:
            res = [paths[i]]
            time_spand = paths_info[i][3]
            max_list[tmp_list[0]] = paths_info[i][tmp_list[0]]
            max_list[tmp_list[1]] = paths_info[i][tmp_list[1]]
            max_list[tmp_list[2]] = paths_info[i][tmp_list[2]]
        elif paths_info[i][tmp_list[0]] == max_list[tmp_list[0]]:
            if paths_info[i][tmp_list[1]] < max_list[tmp_list[1]]:
                res = [paths[i]]
                time_spand = paths_info[i][3]
                max_list[tmp_list[0]] = paths_info[i][tmp_list[0]]
                max_list[tmp_list[1]] = paths_info[i][tmp_list[1]]
                max_list[tmp_list[2]] = paths_info[i][tmp_list[2]]
            elif paths_info[i][tmp_list[1]] == max_list[tmp_list[1]]:
                if paths_info[i][tmp_list[2]] < max_list[tmp_list[2]]:
                    res = [paths[i]]
                    time_spand = paths_info[i][3]
                    max_list[tmp_list[0]] = paths_info[i][tmp_list[0]]
                    max_list[tmp_list[1]] = paths_info[i][tmp_list[1]]
                    max_list[tmp_list[2]] = paths_info[i][tmp_list[2]]
                elif paths_info[i][tmp_list[2]] == max_list[tmp_list[2]]:
                    res.append(paths[i])
#     print
    if len(res) == 0:
        text_box.insert("end", "沒有符合條件的路線")
        draw_bg()
        return
    else:
        for p in res:
            print_path(p)

        draw_path(res[0])
        text_box.tag_configure("italic", font=("Arial", 12, "roman"))
        text_box.insert("end", "\n最低乘車時間： " + str(info[4]) + " 分鐘\n", "italic")   
        text_box.insert("end", "實際乘車時間預估： " + str(time_spand) +  " 分鐘\n", "italic")
        text_box.insert("end", "票價：\n", "italic")
        text_box.insert("end", "\t全　　　票： " + str(info[0]) + " 元\n", "italic")
        text_box.insert("end", "\t敬老、愛心： " + str(info[1]) + " 元\n", "italic")  
        text_box.insert("end", "\t臺北市兒童： " + str(info[2]) + " 元\n", "italic")  
        text_box.insert("end", "\t新北市兒童： " + str(info[3]) + " 元\n", "italic")  

    return res

def get_road_for_event(event):
    get_road()

### 視窗本體

In [9]:
import tkinter as tk
from tkinter import ttk
import tkinter.font as tkFont
from PIL import Image, ImageTk

# 路徑及路徑資訊
paths = []
info = []
paths_info = []
        
# 建立視窗
root = tk.Tk()
root.title("捷運路網規畫小程式demo")

# 設定視窗大小和位置
width = 800
height = 600
x_pos = 200
y_pos = 150
root.geometry(f"{width}x{height}+{x_pos}+{y_pos}")
root.minsize(width, height)
root.maxsize(width, height)

# 視窗選單
print_mod = 1
def on_exit():
    root.destroy()

def on_textmod():
    text_box.pack(pady=20)
    text_box.place(x=280, y=35)
    canvas.place_forget()
    get_road()
    
def on_imgmod():
    text_box.place_forget()
    canvas.pack(pady=20)
    canvas.place(x=280, y=20)
    get_road()

menu = tk.Menu(root)
root.config(menu=menu)
file_menu = tk.Menu(menu, tearoff=False, activebackground="blue", activeforeground="white")
view_menu = tk.Menu(menu, tearoff=False, activebackground="blue", activeforeground="white")
menu.add_cascade(label="檔案", menu=file_menu)
file_menu.add_command(label="離開", command=on_exit)
menu.add_cascade(label="顯示", menu=view_menu)
view_menu.add_command(label="文字顯示", command=on_textmod)
view_menu.add_command(label="圖片顯示", command=on_imgmod)

# 起訖點下拉選單
options = list(graph.keys())

label_start = tk.Label(root, text="起始站點：")
label_start.pack(pady=20)
label_start.place(x=20, y=30)
combobox_start = ttk.Combobox(root, values=options, state="readonly")
combobox_start.pack(pady=20)
combobox_start.set("請選擇站點")
combobox_start.place(x=20, y=50)

label_end = tk.Label(root, text="目的站點：")
label_end.pack(pady=20)
label_end.place(x=20, y=80)
combobox_end = ttk.Combobox(root, values=options, state="readonly")
combobox_end.pack(pady=20)
combobox_end.set("請選擇站點")
combobox_end.place(x=20, y=100)


# 文字框
bold_font = tkFont.Font(family="Arial", size=12, weight="bold")
text_box = tk.Text(root, width=50, height=28, font=bold_font)
text_box.pack(pady=20)
text_box.place(x=280, y=35)
text_box.configure(bg="#f0f0f0")

# 圖形框
canvas = tk.Canvas(root, width=600, height=550)
canvas.pack(pady=20)
canvas.place(x=280, y=20)

# 起訖點按鈕
def on_button1_click():
    global paths
    global info
    global paths_info
    
    start_option = combobox_start.get()
    end_option = combobox_end.get()
    text_box.delete("1.0", tk.END)
    if start_option != '請選擇站點' and  end_option != '請選擇站點' and start_option != end_option:
        paths = find_all_paths(graph, start_option, end_option)
        info = csv_data[combobox_start.current()+1][combobox_end.current()+1].split("/")
        paths_info = get_paths_info(paths)
        
    get_road()

button1 = tk.Button(root, text="確認", command=on_button1_click)
button1.pack(pady=20)
button1.place(x=20, y=140)


# 路線篩選勾勾
checkbox_var_BR = tk.IntVar(value=1)
checkbox_var_R = tk.IntVar(value=1)
checkbox_var_G = tk.IntVar(value=1)
checkbox_var_O = tk.IntVar(value=1)
checkbox_var_BL = tk.IntVar(value=1)
checkbox_var_Y = tk.IntVar(value=1)
checkbox_BR = tk.Checkbutton(root, text="BR 文湖線", variable = checkbox_var_BR, command=get_road)
checkbox_BR.pack(pady=20)
checkbox_BR.place(x=15, y=450)
checkbox_R = tk.Checkbutton(root, text="R 淡水信義線", variable = checkbox_var_R, command=get_road)
checkbox_R.pack(pady=20)
checkbox_R.place(x=15, y=470)
checkbox_G = tk.Checkbutton(root, text="G 松山新店線", variable = checkbox_var_G, command=get_road)
checkbox_G.pack(pady=20)
checkbox_G.place(x=15, y=490)
checkbox_O = tk.Checkbutton(root, text="O 中和新盧線", variable = checkbox_var_O, command=get_road)
checkbox_O.pack(pady=20)
checkbox_O.place(x=15, y=510)
checkbox_BL = tk.Checkbutton(root, text="BL 板南線", variable = checkbox_var_BL, command=get_road)
checkbox_BL.pack(pady=20)
checkbox_BL.place(x=15, y=530)
checkbox_Y = tk.Checkbutton(root, text="Y 環狀線", variable = checkbox_var_Y, command=get_road)
checkbox_Y.pack(pady=20)
checkbox_Y.place(x=15, y=550)


# 優先選單
label_sort = tk.Label(root, text="選擇優先級：")
label_sort.pack(pady=20)
label_sort.place(x=20, y=400)

combobox_sort = ttk.Combobox(root, values=["時間優先", "最少轉乘優先", "最少站數優先"], state="readonly")
combobox_sort.pack(pady=20)
combobox_sort.place(x=20, y=420)
combobox_sort.bind("<<ComboboxSelected>>", get_road_for_event)
combobox_sort.current(0)

draw_bg()

## 執行

In [10]:
root.mainloop()

# 基本執行階段

### 使用者輸入

In [11]:
import ipywidgets as widgets
start_node = ""
end_node = ""

options = list(graph.keys())
options.insert(0, "請選擇站點")
dropdown_start = widgets.Dropdown(options=options)
dropdown_end = widgets.Dropdown(options=options)

print("起始站點：")
display(dropdown_start)

print("目的站點：")
display(dropdown_end)


button = widgets.Button(description='確認')
def on_button_click(button):
    global start_node
    global end_node
    start_node = dropdown_start.value
    end_node = dropdown_end.value
    if start_node != '請選擇站點' and  end_node != '請選擇站點' and start_node != end_node:
        dropdown_start.disabled = True
        dropdown_end.disabled = True
        button.layout.visibility = 'hidden'

button.on_click(on_button_click)
display(button)

起始站點：


Dropdown(options=('請選擇站點', 'BR01 動物園', 'BR02 木柵', 'BR03 萬芳社區', 'BR04 萬芳醫院', 'BR05 辛亥', 'BR06 麟光', 'BR07 六張犁', …

目的站點：


Dropdown(options=('請選擇站點', 'BR01 動物園', 'BR02 木柵', 'BR03 萬芳社區', 'BR04 萬芳醫院', 'BR05 辛亥', 'BR06 麟光', 'BR07 六張犁', …

Button(description='確認', style=ButtonStyle())

### 資料建立

In [12]:
paths = find_all_paths(graph, start_node, end_node)

# 0:全票票價; 1:敬老、愛心; 2:臺北市兒童; 3:新北市兒童; 4:乘車時間
info = csv_data[dropdown_start.index][dropdown_end.index].split("/")

# 0:站數; 1:轉乘數; 2:經過路線; 3:所需時間
paths_info = get_paths_info(paths)

IndexError: list index out of range

### 打印成果

In [ ]:
from colorama import init, Fore, Style
def __print_station_color(station, endchar):
    if extract_english(station) == "BR":
        coffee = f"\x1b[38;2;{101};{67};{33}m"
        print(coffee + station, end = "")
    elif extract_english(station) == "R" or extract_english(station) == "R A":
        print(Fore.RED + station, end = "")
    elif extract_english(station) == "G" or extract_english(station) == "G A":
        print(Fore.GREEN + station, end = "")
    elif extract_english(station) == "O":
        orange = f"\x1b[38;2;{255};{165};{0}m"
        print(orange + station, end = "")
    elif extract_english(station) == "BL":
        print(Fore.BLUE + station, end = "")
    elif extract_english(station) == "Y":
        print(Fore.YELLOW + station, end = "")
    
    print(Style.RESET_ALL + endchar, end = "")
    return 0

def print_path(list):
    if len(list) == 2:
        __print_station_color(list[0], " -> ")
        __print_station_color(list[1], "\n")
        return
    
    if extract_chinese(list[0]) == extract_chinese(list[1]):
        __print_station_color(list[1], " -> ")
    else:
        __print_station_color(list[0], " -> ")
    
    for i in range(2, len(list)-1):
        if extract_chinese(list[i-1]) == extract_chinese(list[i]):
            __print_station_color(list[i-1].split(" ")[0] + "/", "")
            __print_station_color(list[i], " -> ")
    
    if extract_chinese(list[len(list)-2]) == extract_chinese(list[len(list)-1]):
        __print_station_color(list[len(list)-2], "\n")
    else:
        __print_station_color(list[len(list)-1], "\n")
        
    return
    
    
        
for p in range(0, len(paths)):
    print(paths[p], paths_info[p])
    print_path(paths[p])

print()
print('\033[1m'"最低乘車時間:", info[4], "分鐘"'\033[0m')
print('\033[1m'"實際乘車時間預估:", 0, "分鐘"'\033[0m')

print()
print('\033[1m'"票價："'\033[0m')
print("\t全票：",info[0],"元")
print("\t敬老、愛心：",info[1],"元")
print("\t臺北市兒童",info[2],"元")
print("\t新北市兒童：",info[3],"元")

[''] [11, 1, ['BR', 'R'], 22]


IndexError: list index out of range